In [2]:
using Plots, CSV, DataFrames, Distributions, MKL, LinearAlgebra, BenchmarkTools, MAT, Printf

In [13]:
import Pkg
Pkg.add("MAT")
Pkg.add("Printf")

    Updating registry at `C:\Users\user\.julia\registries\General.toml`
   Resolving package versions...
   Installed HDF5_jll ──────── v1.12.2+2
   Installed BufferedStreams ─ v1.1.0
   Installed MAT ───────────── v0.10.3
   Installed HDF5 ──────────── v0.16.12
    Updating `C:\Users\user\.julia\environments\v1.8\Project.toml`
  [23992714] + MAT v0.10.3
    Updating `C:\Users\user\.julia\environments\v1.8\Manifest.toml`
  [e1450e63] + BufferedStreams v1.1.0
  [f67ccb44] + HDF5 v0.16.12
  [23992714] + MAT v0.10.3
  [0234f1f7] + HDF5_jll v1.12.2+2
    Building HDF5 → `C:\Users\user\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\19effd6b5af759c8aaeb9c77f89422d3f975ab65\build.log`
Precompiling project...
  ✓ BufferedStreams
  ✓ HDF5_jll
  ✓ HDF5
  ✓ MAT
  4 dependencies successfully precompiled in 14 seconds. 171 already precompiled.
   Resolving package versions...
    Updating `C:\Users\user\.julia\environments\v1.8\Project.toml`
  [de0858da] + Printf
  No Changes to `C:\User

In [ ]:
f=rand(1,1)

In [ ]:
B = [1.5 2 -4; 2 -1 -3; -4 -3 5]
factorize(B)

In [ ]:
U = UniformScaling(2)
U[1:2,1:2]

In [ ]:
A=[0 2;1 3]
factorize(A)

In [ ]:
A=[1 3;0 2]
factorize(A)

In [ ]:
A = reshape(collect(1:16), (2, 2, 2, 2))

In [ ]:
A = [4 3; 6 3]
F = lu(A)
F.L * F.U == F.P*A


In [ ]:
fill(2, (5,6))

In [ ]:
c=ones(333,1)
d=ones(1,333)
e=ones(333,333)
mul!(e,c,d)
size(e)


In [ ]:

    f_mo_hat_new=copy(f_mo_hat);
    global w = f_mo_hat_new./g / sum(f_mo_hat_new./g);
    tempC = similar(fdensity);
    mul!(tempC,ones(num_obs,1),f_mo_hat_new)
    print(tempC)
    temptop = (tempC.*fdensity);
    mul!(tempC,ones(num_obs,1),w)
    tempbottom = (sum(tempC.*fdensity,dims=2)*ones(1,nS));
    temp_a  = findall(x-> x==0,tempbottom); 
    #temp_max = 10*maximum(tempbottom);
    #tempbottom[temp_a] .= temp_max;
    #temp_min = 0.01*minimum(tempbottom);
    #tempbottom[temp_a] .= temp_min;
    tempbottom[temp_a] .= 0.0001;
    global f_mo_hat = mean( temptop./ tempbottom, dims = 1);

In [17]:
type Coordinate
    x::Float64
    y::Float64
end

c = Coordinate(3, 4.0)
print(c)

LoadError: syntax: extra token "Coordinate" after end of expression

In [30]:
prog = "1+2"
ex = Meta.parse(prog)
ex = quote
    1+2
end;

In [6]:
a = [[1,2,3] [4,5,6]]
typeof(a)
b=[a]
typeof(b)
print(b)
c = [[1,2,3] [4,5,6] [7,8,9]]
print(c)
d= [[1 2 3] [4 5 6]]
a = [[1,2,3] [4,5,6]]

[[1 4; 2 5; 3 6]][1 4 7; 2 5 8; 3 6 9]

3×2 Matrix{Int64}:
 1  4
 2  5
 3  6

In [8]:
a = 5
while a > 2
 global a
 println("a: $a")
 b = a
 while b > 2
 println("b: $b")
 b -= 1
 end
 a -= 1
end

a: 5
b: 5
b: 4
b: 3
a: 4
b: 4
b: 3
a: 3
b: 3


In [10]:
a = 5
while a > 2
 global a
 println("a: $a")
 b = a
 while b > 2
 println("b: $b")
 b -= 1
 end
 a -= 1
end

a: 5
b: 5
b: 4
b: 3
a: 4
b: 4
b: 3
a: 3
b: 3


In [11]:
t = (1,2.5,"a")
@time typeof(t)

  0.000002 seconds


Tuple{Int64, Float64, String}

In [7]:
# # Chapter 1

# ### Inspect the generated code for the power function.


# For integers:
@code_native 3^2

# And for floats
@code_native 3.5^2


	.text
	.file	"^"
	.globl	"julia_^_1372"                  # -- Begin function julia_^_1372
	.p2align	4, 0x90
	.type	"julia_^_1372",@function
"julia_^_1372":                         # @"julia_^_1372"
; ┌ @ intfuncs.jl:299 within `^`
	.cfi_startproc
# %bb.0:                                # %top
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset %rbp, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register %rbp
	subq	$32, %rsp
	movabsq	$j_power_by_squaring_1374, %rax
	callq	*%rax
	addq	$32, %rsp
	popq	%rbp
	retq
.Lfunc_end0:
	.size	"julia_^_1372", .Lfunc_end0-"julia_^_1372"
	.cfi_endproc
; └
                                        # -- End function
	.section	".note.GNU-stack","",@progbits
	.text
	.file	"^"
	.section	.rodata.cst8,"aM",@progbits,8
	.p2align	3                               # -- Begin function julia_^_1408
.LCPI0_0:
	.quad	0x3ff0000000000000              # double 1
	.text
	.globl	"julia_^_1408"
	.p2align	4, 0x90
	.type	"julia_^_1408",@function
"julia_^_1408":                         # @"j

In [19]:
# # Chapter 1

# ## Using Types
using BenchmarkTools

# Declare type of function argument
iam(x::Integer) = "an integer"
iam(x::String) = "a string"
iam(x::Float) = "a float"

function addme(a, b)
  #Declare type of local variable x
  x::Int64 = 2
  #Type of variable y will be inferred
  y = (a+b) / x
  return y
end

iam(3.3)

LoadError: UndefVarError: Float not defined

In [3]:
function string_zeros(s::AbstractString)
    n=1000_000
    x = s=="Int64" ?
        Vector{Int64}(undef,n) :
        Vector{Float64}(undef, n)
    for i in 1:length(x)
        x[i] = 0
    end
    return x
end

@btime string_zeros("Int64");


function string_zeros_stable(s::AbstractString)
    n = 1000_000
    x = s=="Int64" ?
        Vector{Int64}(undef, n) :
        Vector{Float64}(undef, n)
    return fill_zeros(x)
end

function fill_zeros(x)
    for i in 1:length(x)
        x[i] = 0
    end
    return x
end

  1.463 ms (2 allocations: 7.63 MiB)


fill_zeros (generic function with 1 method)

In [15]:
greet = "Hello"
whom = "World"
"$greet, $(whom)!"
"1 + 2 = $(1) + 2"

LoadError: cannot document the following expression:

"1 + 2 = $(1) + 2"


In [17]:
#Types in storage
## Arrays

a = Int64[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
b = Number[1,2,3,4,5,6,7,8,9,10]

function arr_sumsqr(x::Array{T}) where T <: Number
    r = zero(T)
    for i = 1:length(x)
        r = r + x[i] ^ 2
    end
    return r
end

 @btime arr_sumsqr($a)
 @btime arr_sumsqr($b)

  10.800 ns (0 allocations: 0 bytes)
  392.537 ns (0 allocations: 0 bytes)


385

In [23]:
t1 = [1,2]
t2 = push!(t1,3)
print(t2)
t1 === t2

[1, 2, 3]

true

In [28]:
t1 = [1,2]
t2 = t1[2:end]
t1=[1,2,3]
print(t2)
print(t1 == t2)
print(t1)

[2]false[1, 2, 3]

In [42]:
t3=[1,2,3]
function badaboom(t)
    t3 = [1,2]
end
t4 =badaboom(t3)
print(t4)
print(t3)

[1, 2][1, 2, 3]